In [2]:
import findspark
findspark.init()
import pyspark
myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession.builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()

In [3]:
import os

popRddBin = spark.sparkContext.binaryFiles(os.path.join("data","경기도 의정부시_인구현황_20210910.csv"))

In [4]:
_my = popRddBin.map(lambda x :x[1].decode('euc-kr'))

In [5]:
_my.take(1)

['행정기관,인구수(계),인구수(남),인구수(여),구성비(계),구성비(남),구성비(여),성비,세대수,세대당인구,관리기관명,관리부서명,부서전화번호,데이터기준일자\r\n의정부1동,32292,16538,15754,6.97,3.57,3.4,104.98,19998,1.61,의정부시,민원여권과,031-828-2466,2021-09-10\r\n의정부2동,31380,15608,15772,6.77,3.37,3.4,98.96,16410,1.91,의정부시,민원여권과,031-828-2466,2021-09-10\r\n호원1동,36124,17595,18529,7.8,3.8,4,94.96,15653,2.31,의정부시,민원여권과,031-828-2466,2021-09-10\r\n호원2동,34957,16923,18034,7.54,3.65,3.89,93.84,13683,2.55,의정부시,민원여권과,031-828-2466,2021-09-10\r\n장암동,20314,9714,10600,4.38,2.1,2.29,91.64,8604,2.36,의정부시,민원여권과,031-828-2466,2021-09-10\r\n신곡1동,43159,21205,21954,9.31,4.58,4.74,96.59,17990,2.4,의정부시,민원여권과,031-828-2466,2021-09-10\r\n신곡2동,47852,23232,24620,10.33,5.01,5.31,94.36,19218,2.49,의정부시,민원여권과,031-828-2466,2021-09-10\r\n송산1동,42817,21276,21541,9.24,4.59,4.65,98.77,18811,2.28,의정부시,민원여권과,031-828-2466,2021-09-10\r\n송산2동,33565,16601,16964,7.24,3.58,3.66,97.86,13216,2.54,의정부시,민원여권과,031-828-2466,2021-09-10\r\n송산3동,46892,22772,24120,10.12,4.91,5.21,94.41,17926,2.62,의정부시,민원여권과,031-828-2466

In [6]:
popList = _my.map(lambda x: x.split()).take(3)
popList[0][1]

'의정부1동,32292,16538,15754,6.97,3.57,3.4,104.98,19998,1.61,의정부시,민원여권과,031-828-2466,2021-09-10'

# DataFrame

In [7]:
popDf = spark\
            .read.option("charset", "euc-kr")\
            .option("header", "true")\
            .csv(os.path.join("data","경기도 의정부시_인구현황_20210910.csv"))

In [8]:
popDf.show(5)

+---------+----------+----------+----------+----------+----------+----------+------+------+----------+----------+----------+------------+--------------+
| 행정기관|인구수(계)|인구수(남)|인구수(여)|구성비(계)|구성비(남)|구성비(여)|  성비|세대수|세대당인구|관리기관명|관리부서명|부서전화번호|데이터기준일자|
+---------+----------+----------+----------+----------+----------+----------+------+------+----------+----------+----------+------------+--------------+
|의정부1동|     32292|     16538|     15754|      6.97|      3.57|       3.4|104.98| 19998|      1.61|  의정부시|민원여권과|031-828-2466|    2021-09-10|
|의정부2동|     31380|     15608|     15772|      6.77|      3.37|       3.4| 98.96| 16410|      1.91|  의정부시|민원여권과|031-828-2466|    2021-09-10|
|  호원1동|     36124|     17595|     18529|       7.8|       3.8|         4| 94.96| 15653|      2.31|  의정부시|민원여권과|031-828-2466|    2021-09-10|
|  호원2동|     34957|     16923|     18034|      7.54|      3.65|      3.89| 93.84| 13683|      2.55|  의정부시|민원여권과|031-828-2466|    2021-09-10|
|   장암동|     20314|      9714|     10600

In [11]:
popDf = spark\
            .read.option("charset", "euc-kr")\
            .option("header", "true")\
            .csv(os.path.join("data","제주특별자치도 서귀포시_고령화비율및노령화지수현황_20210831.csv"))

In [12]:
popDf.show(5)

+------+---------------+----------------+---------------+----------+----------+--------------+
|연도별|서귀포시 인구수|65세이상 인구수 |14세이하 인구수|고령화비율|노령화지수|데이터기준일자|
+------+---------------+----------------+---------------+----------+----------+--------------+
|  2012|         154057|           25826|          22861|     16.76|    112.97|    2021-08-31|
|  2013|         155641|           26936|          22393|     17.31|    120.29|    2021-08-31|
|  2014|         158512|           27877|          22058|     17.59|    126.38|    2021-08-31|
|  2015|         164519|           28979|          22362|     17.61|    129.59|    2021-08-31|
|  2016|         170932|           30030|          23044|     17.57|    130.32|    2021-08-31|
+------+---------------+----------------+---------------+----------+----------+--------------+
only showing top 5 rows



In [43]:
%%writefile src/ds3_popCsvRead.py
#!/usr/bin/env python3
# -*- coding: UTF-8 -*-
import os
import findspark
findspark.init()
import pyspark

def doIt():
    print ("---------RESULT-----------")
    popDf = spark\
                .read.option("charset", "euc-kr")\
                .option("header", "true")\
                .csv(os.path.join("data","경기도 의정부시_인구현황_20210910.csv"))
    popDf.show(5)
    agedDf = spark\
                .read.option("charset", "euc-kr")\
                .option("header", "true")\
                .csv(os.path.join("data","제주특별자치도 서귀포시_고령화비율및노령화지수현황_20210831.csv"))
    agedDf.show(5)

if __name__ == "__main__":
    myConf=pyspark.SparkConf()
    spark = pyspark.sql.SparkSession.builder\
        .master("local")\
        .appName("myApp")\
        .config(conf=myConf)\
        .getOrCreate()
    doIt()
    spark.stop()

Overwriting src/ds3_popCsvRead.py


In [44]:
!spark-submit src/ds3_popCsvRead.py

---------RESULT-----------
+---------+----------+----------+----------+----------+----------+----------+------+------+----------+----------+----------+------------+--------------+
| 행정기관|인구수(계)|인구수(남)|인구수(여)|구성비(계)|구성비(남)|구성비(여)|  성비|세대수|세대당인구|관리기관명|관리부서명|부서전화번호|데이터기준일자|
+---------+----------+----------+----------+----------+----------+----------+------+------+----------+----------+----------+------------+--------------+
|의정부1동|     32292|     16538|     15754|      6.97|      3.57|       3.4|104.98| 19998|      1.61|  의정부시|민원여권과|031-828-2466|    2021-09-10|
|의정부2동|     31380|     15608|     15772|      6.77|      3.37|       3.4| 98.96| 16410|      1.91|  의정부시|민원여권과|031-828-2466|    2021-09-10|


21/09/23 14:14:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
21/09/23 14:14:35 INFO SparkContext: Running Spark version 3.1.2

|  호원1동|     36124|     17595|     18529|       7.8|       3.8|         4| 94.96| 15653|      2.31|  의정부시|민원여권과|031-828-2466|    2021-09-10|
|  호원2동|     34957|     16923|     18034|      7.54|      3.65|      3.89| 93.84| 13683|      2.55|  의정부시|민원여권과|031-828-2466|    2021-09-10|
|   장암동|     20314|      9714|     10600|      4.38|       2.1|      2.29| 91.64|  8604|      2.36|  의정부시|민원여권과|031-828-2466|    2021-09-10|
+---------+----------+----------+----------+----------+----------+----------+------+------+----------+----------+----------+------------+--------------+
only showing top 5 rows

+------+---------------+----------------+---------------+----------+----------+--------------+
|연도별|서귀포시 인구수|65세이상 인구수 |14세이하 인구수|고령화비율|노령화지수|데이터기준일자|
+------+---------------+----------------+---------------+----------+----------+--------------+
|  2012|         154057|           25826|          22861|     16.76|    112.97|    2021-08-31|
|  2013|         155641|           26936|          22393| 


21/09/23 14:14:35 INFO ResourceUtils: ==============================================================
21/09/23 14:14:35 INFO ResourceUtils: No custom resources configured for spark.driver.
21/09/23 14:14:35 INFO ResourceUtils: ==============================================================
21/09/23 14:14:35 INFO SparkContext: Submitted application: myApp
21/09/23 14:14:35 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
21/09/23 14:14:35 INFO ResourceProfile: Limiting resource is cpu
21/09/23 14:14:35 INFO ResourceProfileManager: Added ResourceProfile id: 0
21/09/23 14:14:35 INFO SecurityManager: Changing view acls to: user
21/09/23 14:14:35 INFO SecurityManager: Changing modify acls to: user
21/09/23 14:14:35 INFO SecurityManager: Changin